In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = False

In [3]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Archive/Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics", "skyvuw"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

  0%|          | 0/5 [00:00<?, ?it/s]

0                                                     Archive/Gabrielle/
1                                               Archive/Gabrielle/Orders
2                                          Archive/Gabrielle/Orders/AOIs
3        Archive/Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4             Archive/Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                      ...                               
73381                               skyvuw/SN2975/SN2975_TILE_LAYOUT.dbf
73382                             skyvuw/SN2975/SN2975_PHOTO_CENTRES.dbf
73383                             skyvuw/SN2975/SN2975_PHOTO_CENTRES.shx
73384                               skyvuw/SN2975/SN2975_TILE_LAYOUT.prj
73385                               skyvuw/SN2975/SN2975_TILE_LAYOUT.shp
Length: 319584, dtype: object

In [4]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}.*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29918,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29919,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29922,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29923,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29930,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
111717,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
111722,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
112823,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
112824,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_13FEB2021.shp


In [5]:
image_extensions = (".jpg", ".jp2", ".tif", ".TIF", ".png")

In [6]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith(image_extensions)]
images

1171      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
1172      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
1174        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
1177        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
1191        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
113056            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
113062            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
113065            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
113066            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
113069            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Length: 2667, dtype: object

In [7]:
Gabrielle_images = filelist[filelist.str.contains("Gabrielle") & filelist.str.endswith(image_extensions)]
Gabrielle_images

289                                                                                                                                                    Archive/Gabrielle/Pressure sensors/Emma's field notes.jpg
314                                                                                                                  Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4737_Northland_New-Zealand_2023-03-03_2250.png
315                                                                                                                  Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4794_Northland_New-Zealand_2023-03-25_2229.tif
319                                                                                                                    Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4794_Waikato_New-Zealand_2023-08-25_2133.tif
320                                                                                                                    Archive/Gabrielle/Imagery/skyfiworking/SkyFi_

In [8]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    else:
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=images, processor=fuzz_preprocess)
        return match, score

df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

/tmp/ipykernel_621510/4132008065.py:24: TqdmWarning: Iterable length 3009 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))


  0%|          | 0/3009 [00:00<?, ?it/s]

Perfect matches: 1896
Imperfect matches: 1113


In [9]:
df = df.sort_values("match_score")
df

,filename,matched_image,match_score
30921,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054
30996,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529
30834,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061
33592,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111
29982,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111
...,...,...,...
55594,Retrolens/Auckland/LongBay/Shorelines/LongBay_20MAR1988.shp,Retrolens/Auckland/LongBay/Stack/LongBay_20MAR1988_mosaic.jp2,100.000000
17495,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_01JAN2015.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_01JAN2015.tif,100.000000
17498,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_28JAN2022.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_28JAN2022.tif,100.000000
55569,Retrolens/Auckland/LongBay/Shorelines/LongBay_19JAN1973.shp,Retrolens/Auckland/LongBay/Stack/LongBay_19JAN1973_mosaic.jp2,100.000000


In [10]:
def get_shapefile_meta(tup):
    i, row = tup
    filename = row.filename
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return {"Source": source, "n_lines": 0}
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    CPS = ""
    if "CPS" not in shapefile.columns:
        CPS = "No CPS column"
    else:
        CPS = shapefile.CPS.values
        if len(CPS) == 0:
            CPS = "No CPS values"
        elif 9 in CPS:
            CPS = "Has CPS=9"
        elif 0 in CPS:
            CPS = "Has CPS=0"
        elif None in CPS:
            CPS = "Has None CPS"
        elif pd.isna(CPS).any():
            CPS = "Has NA CPS"
        else:
            CPS = "OK"
    return {
        "Source": source,
        "n_lines": n_lines,
        "CPS": CPS,
        "Photoscale": "Photoscale" in shapefile.columns,
        "Pixel_Er": "Pixel_Er" in shapefile.columns
    }

shp_meta = pd.DataFrame(process_map(get_shapefile_meta, df.iterrows(), total=len(df)), index=df.index)
df = pd.concat([df, shp_meta], axis=1)
df

  0%|          | 0/3009 [00:00<?, ?it/s]

Retrolens/Waikato/OpitoBay/Shorelines/Opito_OtamaBay_14FEB2023.shp has ambiguous sources: ['PLE' 'MAX']
Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Waikato/Whangamata/Shorelines/Whangamata_14FEB2023.shp has ambiguous sources: ['MAX' 'PLE']
Retrolens/WestCoast/Westport/Shorelines/Westport_15JAN1988.shp has ambiguous sources: ['Rl' 'RL']
MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines/PareoraRiver_Timaru_11FEB2015.shp has ambiguous sources: ['MAX' None]
MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines/PareoraRiver_Timaru_16MAR2012.shp has ambiguous sources: ['MAX' None]
MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines/PareoraRiver_Timaru_30JAN2021.shp has ambiguous sources: ['MAX' None]MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines/PareoraRiver_Timaru_09APR2010.shp has ambiguous sources: ['MAX' None]

MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Shorelines

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30921,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30996,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30834,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33592,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29982,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
55594,Retrolens/Auckland/LongBay/Shorelines/LongBay_20MAR1988.shp,Retrolens/Auckland/LongBay/Stack/LongBay_20MAR1988_mosaic.jp2,100.000000,RL,1,OK,True,True
17495,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_01JAN2015.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_01JAN2015.tif,100.000000,MAX,8,OK,True,True
17498,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_28JAN2022.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_28JAN2022.tif,100.000000,MAX,9,OK,True,True
55569,Retrolens/Auckland/LongBay/Shorelines/LongBay_19JAN1973.shp,Retrolens/Auckland/LongBay/Stack/LongBay_19JAN1973_mosaic.jp2,100.000000,RL,2,OK,True,True


In [11]:
df.CPS.value_counts()

CPS
OK               2945
No CPS column      40
Has CPS=0          15
No CPS values       8
Has None CPS        1
Name: count, dtype: int64

In [12]:
df[(df.CPS!="OK") & (df.n_lines > 0)][["filename", "Source", "n_lines", "CPS"]].sort_values(by=["CPS", "filename"]).to_csv(prefix+"Nick/CPS_issues.csv", index=False)

In [13]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30921,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30996,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30834,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33592,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29982,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
10325,MaxarImagery/HighFreq/BayOfPlenty/PortOhope/Shorelines/PortOhope_21JUNE2010.shp,MaxarImagery/HighFreq/BayOfPlenty/PortOhope/Stack/PortOhope_21JUNE2010.tif,100.000000,MAX,2,OK,True,True
91175,Retrolens/Northland/MataiBay/Shorelines/MataiBay_20FEB1984.shp,Retrolens/Northland/MataiBay/Stack/MataiBay_20FEB1984.tif,100.000000,RL,15,OK,True,True
91189,Retrolens/Northland/MataiBay/Shorelines/MataiBay_14AUG1944.shp,Retrolens/Northland/MataiBay/Stack/MataiBay_14AUG1944.tif,100.000000,RL,3,OK,True,True
93121,Retrolens/WestCoast/WoodpeckerBay/Shorelines/WoodpeckerBay_17APR1976_mosaic.shp,Retrolens/WestCoast/WoodpeckerBay/Stack/WoodpeckerBay_17APR1976_mosaic.jp2,100.000000,RL,3,OK,True,True


In [14]:
failures = df[(df.match_score < 100) & (df.Source != "LDS") & (df.n_lines > 0)]
try:
  failures.to_csv(prefix+"Nick/failures.csv", index=False)
except:
  print("Can't write Nick/failures.csv")
failures

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30921,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30996,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30834,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33592,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29982,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
90265,Retrolens/Northland/Mangawhai/Shorelines/Mangawhai_10JAN1983_CF.shp,Retrolens/Northland/Mangawhai/Stack/Mangawhai_10JAN1983_mosaic.jp2,94.736842,RL,2,OK,False,False
90293,Retrolens/Northland/Mangawhai/Shorelines/Mangawhai_20MAY1963_CF.shp,Retrolens/Northland/Mangawhai/Stack/Mangawhai_20MAY1963_mosaic.jp2,94.736842,RL,4,OK,False,False
51948,Retrolens/Auckland/Taurangaruru/Shorelines/Taurangaruru_29APR1942.shp,Retrolens/Auckland/Taurangaruru/Stack/Taurangruru_29APR1942.jp2,97.560976,RL,11,OK,True,True
35571,MaxarImagery/HighFreq/Northland/Mimiwhangata/Shorelines/Mimiwhangata_04JUN2008.shp,MaxarImagery/HighFreq/Northland/Mimiwhangata/Imagery/Stack/Mimiwhangata_04JUNE2008.tif,97.560976,MAX,7,OK,True,True


In [15]:
failures[(failures.Source == "RL") & ~failures.filename.str.contains("Waikato")]

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
64220,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_09APR1965.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_09APR1965_mosaic.jp2,88.372093,RL,9,OK,True,True
64229,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_08FEB1952.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_08FEB1952_mosaic.jp2,88.372093,RL,9,OK,True,True
64190,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_27MAR1938.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_27MAR1938_mosaic.jp2,88.888889,RL,12,OK,True,True
64221,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_17FEB1968.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_17FEB1968_mosaic.jp2,88.888889,RL,4,OK,True,True
64215,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_31JAN1980.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_31JAN1980_mosaic.jp2,88.888889,RL,12,OK,True,True
64213,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_11OCT1950.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_11OCT1950_mosaic.jp2,88.888889,RL,6,OK,True,True
64203,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_29OCT1972.shp,Retrolens/Tasman/TakakaRiver_North/Stack/TakakaRiver_29OCT1972_mosaic.jp2,88.888889,RL,12,OK,True,True
64212,Retrolens/Tasman/TakakaRiver_North/Shorelines/TakakaRiver_North_21FEB2002.shp,MaxarImagery/HighFreq/Tasman/TakakaRiver_North/Imagery/Stack/TakakaRiver_North_16FEB2003.tif,92.000000,RL,10,OK,True,True
56649,Retrolens/Auckland/Muriwai2/Shorelines/Muriwai2_28SEP1968.shp,Retrolens/Auckland/Muriwai2/Stack/Muriwai3_28SEP1968_mosaic.png,94.117647,RL,2,OK,True,True
90308,Retrolens/Northland/Mangawhai/Shorelines/Mangawhai_06JAN1978_CF.shp,Retrolens/Northland/Mangawhai/Stack/Unclipped/Mangawhai_06JAN1978_mosaic.jp2,94.444444,RL,1,OK,False,False


In [16]:
# https://www.waikatocoastaldatabase.org.nz/CoastalDataSet/Details/58
WRAPS = df.filename[df.filename.str.contains("01JAN2002")].to_frame()
WRAPS["Pixel_ER"] = .5
WRAPS["Photoscale"] = 40000
WRAPS.to_csv(prefix+"Nick/WRAPS.csv", index=False)
WRAPS

,filename,Pixel_ER,Photoscale
34424,Retrolens/Waikato/CrayfishPoint/Shorelines/CrayfishPoint_01JAN2002.shp,0.5,40000
33570,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_01JAN2002.shp,0.5,40000
35164,Retrolens/Waikato/Miranda/Shorelines/Miranda_01JAN2002.shp,0.5,40000
34730,Retrolens/Waikato/TekahaPoint/Shorelines/TeKahaPoint_01JAN2002.shp,0.5,40000
33243,Retrolens/Waikato/RuapukeBeach/Shorelines/RuapukeBeach_01JAN2002.shp,0.5,40000
36884,Retrolens/Waikato/AwakinoNorth/Shorelines/AwakinoNorth_01JAN2002.shp,0.5,40000
36369,Retrolens/Waikato/Aotea/Shorelines/Aotea_01JAN2002.shp,0.5,40000
34511,Retrolens/Waikato/NukuhakariBay/Shorelines/NukuhakariBay_01JAN2002.shp,0.5,40000
35050,Retrolens/Waikato/Awakino/Shorelines/Awakino_01JAN2002.shp,0.5,40000
32050,Retrolens/Waikato/PotahiPoint/Shorelines/PotahiPoint_01JAN2002.shp,0.5,40000


In [17]:
RL_Waikato = failures[failures.filename.str.contains("Waikato") & (failures.Source == "RL") & ~failures.filename.isin(WRAPS.filename)]
RL_Waikato

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
34324,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,True,True
37391,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,True,True
33148,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,RL,2,OK,True,True
33143,Retrolens/Waikato/Raglan/Shorelines/Raglan_01MAY1974.shp,Retrolens/Northland/Rangiputa/Stack/Rangiputa_26MAY1947.tif,62.500000,RL,1,OK,True,True
33137,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,RL,4,OK,True,True
...,...,...,...,...,...,...,...,...
33946,Retrolens/Waikato/TeMataBay/Shorelines/TeMataBay_09FEB1971.shp,Retrolens/Northland/MataiBay/Stack/MataiBay_11FEB1971.tif,82.352941,RL,3,OK,True,True
36333,Retrolens/Waikato/Aotea/Shorelines/Aotea_12SEP1961.shp,Retrolens/Auckland/Orewa/Stack/Orewa_12SEP1961.jp2,85.714286,RL,1,OK,True,True
35617,Retrolens/Waikato/Pauanui_Tairua/Shorelines/Pauanui_Tairua_09FEB1971.shp,MaxarImagery/HighFreq/Waikato/Pauanui_Tairua/Imagery/Stack/Pauanui_Tairua_09FEB2019.tif,90.476190,RL,4,OK,True,True
34171,Retrolens/Waikato/PortWaikato/Shorelines/PortWaikato_29MAR2012.shp,MaxarImagery/HighFreq/Waikato/PortWaikato/Imagery/Stack/PortWaikato_09MAR2016.tif,92.307692,RL,1,OK,True,True


In [18]:
def get_survey_number(filename, DSASDate, year):
    # Find CSV for AOI
    bits = filename.split("/")
    terminator = min(
        bits.index("Stack") if "Stack" in bits else 1024,
        bits.index("Shorelines") if "Shorelines" in bits else 1024,
    )
    bits = bits[:terminator]
    csv_path_pattern = f"{prefix}{'/'.join(bits)}/*.csv"
    csv_candidates = glob(csv_path_pattern)
    if len(csv_candidates) == 0:
        raise ValueError(f"No CSV found for {csv_path_pattern}")
    elif len(csv_candidates) > 1:
        print(f"Ambiguous CSVs: {csv_candidates}")
        csv_candidates = [csv_candidates[0]]
    assert len(csv_candidates) == 1
    csv_filename = csv_candidates[0]
    try:
        csv = pd.read_csv(csv_filename, encoding="cp1252")
    except UnicodeDecodeError:
        # Excel file saved with .csv extension
        print(f"{csv_filename} is actually an Excel file")
        csv = pd.read_excel(csv_filename)
        csv.Date = csv.Date.astype(str)

    # Fuzzy string match dates. Lots of typos.
    matched_date, score, index = rapidfuzz.process.extractOne(query=DSASDate, choices=csv.Date.unique())
    if score < 80:
        # Terrible match score, just use year
        matched_date, score, index = rapidfuzz.process.extractOne(query=year, choices=csv.Date.unique())
        print(f"Matching {year} to {matched_date} with score {score} for {csv_filename}")
    elif DSASDate != matched_date:
        print(f"Matching {DSASDate} to {matched_date} with score {score} for {csv_filename}")
    if "RMSE" in csv.columns:
        filtered_csv = csv[(csv.Date == matched_date) & ~csv.RMSE.isna()]
    else:
        filtered_csv = csv[(csv.Date == matched_date)]
    surveys = filtered_csv.Survey.unique()
    if len(surveys) == 0:
        print(f"No surveys found for {csv_filename} for date {DSASDate}, removing RMSE filter")
        filtered_csv = csv[(csv.Date.str.contains(matched_date))]
        surveys = filtered_csv.Survey.unique()
        print(f"Survey now: {surveys}")
        if len(surveys) == 0:
            print(f"Still no surveys found for {csv_filename} for date {matched_date}, reducing date filter just to year from filename {year}")
            filtered_csv = csv[(csv.Date.str.contains(year))]
            surveys = filtered_csv.Survey.unique()
            print(f"Survey now: {surveys}")
    if len(surveys) > 1:
        #print(f"Ambiguous surveys found for {csv_candidates[0]} for date {DSASDate}: {surveys}")
        pass
    if len(surveys) == 0:
        print(f"Can't find a survey for {filename}")
    return surveys

def process_shapefile(filename):
    year = re.search(r'(\d{4})', filename).group(1)
    shapefile = gpd.read_file(prefix + filename)
    date = shapefile.Date.unique()[0]
    DSASDate = pd.to_datetime(date).strftime("%d/%m/%Y")
    survey_numbers = get_survey_number(filename, DSASDate, year)
    #print(filename, survey_numbers)
    for survey_number in survey_numbers:
        try:
            survey_tiles = gpd.read_file(f"{prefix}/skyvuw/SN{survey_number}/SN{survey_number}_TILE_LAYOUT.shp")
            GSD = float(survey_tiles.GSD.dropna().astype(str).str.strip("m").unique()[0])
            #print(f"{survey_number}={GSD}")
            return GSD
        except Exception as e:
            print(f"{survey_number} {e}")
    return None

RL_Waikato["Pixel_ER"] = RL_Waikato.filename.progress_apply(process_shapefile)
RL_Waikato

  0%|          | 0/203 [00:00<?, ?it/s]

Matching 2013 to 22/03/1944 with score 67.5 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/Raglan/Raglan.csv
Matching 01/05/1974 to 1/05/1974 with score 94.73684210526316 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/Raglan/Raglan.csv
Matching 2009 to 26/07/1966 with score 45.0 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/Raglan/Raglan.csv
Matching 2012 to 21/02/1969 with score 60.00000000000001 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/CrayfishPoint/CrayfishPoint.csv
Matching 2012 to 4/01/2001 with score 67.5 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/UrawhitikiPoint/UrawhitikiPoint.csv
Matching 01/05/1974 to 1/05/1974 with score 94.73684210526316 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/CrayfishPoint/CrayfishPoint.csv
253 'GeoDataFrame' object has no attribute 'GSD'
Matching 09/02/1971 to 9/02/1971 with score 94.73684210526316 for ressci201900060-RNC2-Coastal/Retrolens/Waikato/TePuru/TePuru.csv
Matching 2012 to 22/03/1944 with score 60.000000

/tmp/ipykernel_621510/881745768.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RL_Waikato["Pixel_ER"] = RL_Waikato.filename.progress_apply(process_shapefile)


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,Pixel_ER
34324,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,True,True,0.40
37391,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,True,True,0.40
33148,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,RL,2,OK,True,True,0.40
33143,Retrolens/Waikato/Raglan/Shorelines/Raglan_01MAY1974.shp,Retrolens/Northland/Rangiputa/Stack/Rangiputa_26MAY1947.tif,62.500000,RL,1,OK,True,True,0.75
33137,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,RL,4,OK,True,True,0.40
...,...,...,...,...,...,...,...,...,...
33946,Retrolens/Waikato/TeMataBay/Shorelines/TeMataBay_09FEB1971.shp,Retrolens/Northland/MataiBay/Stack/MataiBay_11FEB1971.tif,82.352941,RL,3,OK,True,True,0.40
36333,Retrolens/Waikato/Aotea/Shorelines/Aotea_12SEP1961.shp,Retrolens/Auckland/Orewa/Stack/Orewa_12SEP1961.jp2,85.714286,RL,1,OK,True,True,0.60
35617,Retrolens/Waikato/Pauanui_Tairua/Shorelines/Pauanui_Tairua_09FEB1971.shp,MaxarImagery/HighFreq/Waikato/Pauanui_Tairua/Imagery/Stack/Pauanui_Tairua_09FEB2019.tif,90.476190,RL,4,OK,True,True,0.40
34171,Retrolens/Waikato/PortWaikato/Shorelines/PortWaikato_29MAR2012.shp,MaxarImagery/HighFreq/Waikato/PortWaikato/Imagery/Stack/PortWaikato_09MAR2016.tif,92.307692,RL,1,OK,True,True,NaN


In [19]:
RL_Waikato.Pixel_ER[RL_Waikato.Pixel_ER.isna()] = RL_Waikato.Pixel_ER.mean()
RL_Waikato

/tmp/ipykernel_621510/3977043647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RL_Waikato.Pixel_ER[RL_Waikato.Pixel_ER.isna()] = RL_Waikato.Pixel_ER.mean()


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,Pixel_ER
34324,Retrolens/Waikato/PortJackson/Shorelines/PortJackson_23AUG1971.shp,Retrolens/Northland/TaupiriBay/Stack/TaupiriBay_23AUG1971.tif,61.538462,RL,2,OK,True,True,0.400000
37391,Retrolens/Waikato/Thames/Shorelines/Thames_25JAN1973.shp,Retrolens/Auckland/PakiriBeach_North/Stack/PakiriBeach_North_15JAN1977_mosaic.jp2,62.307692,RL,5,OK,True,True,0.400000
33148,Retrolens/Waikato/Raglan/Shorelines/Raglan_24FEB2013.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,RL,2,OK,True,True,0.400000
33143,Retrolens/Waikato/Raglan/Shorelines/Raglan_01MAY1974.shp,Retrolens/Northland/Rangiputa/Stack/Rangiputa_26MAY1947.tif,62.500000,RL,1,OK,True,True,0.750000
33137,Retrolens/Waikato/Raglan/Shorelines/Raglan_22MAR1944.shp,MaxarImagery/HighFreq/Otago/Aramoana/Imagery/Stack/Aramoana_20MAR2014.tif,62.500000,RL,4,OK,True,True,0.400000
...,...,...,...,...,...,...,...,...,...
33946,Retrolens/Waikato/TeMataBay/Shorelines/TeMataBay_09FEB1971.shp,Retrolens/Northland/MataiBay/Stack/MataiBay_11FEB1971.tif,82.352941,RL,3,OK,True,True,0.400000
36333,Retrolens/Waikato/Aotea/Shorelines/Aotea_12SEP1961.shp,Retrolens/Auckland/Orewa/Stack/Orewa_12SEP1961.jp2,85.714286,RL,1,OK,True,True,0.600000
35617,Retrolens/Waikato/Pauanui_Tairua/Shorelines/Pauanui_Tairua_09FEB1971.shp,MaxarImagery/HighFreq/Waikato/Pauanui_Tairua/Imagery/Stack/Pauanui_Tairua_09FEB2019.tif,90.476190,RL,4,OK,True,True,0.400000
34171,Retrolens/Waikato/PortWaikato/Shorelines/PortWaikato_29MAR2012.shp,MaxarImagery/HighFreq/Waikato/PortWaikato/Imagery/Stack/PortWaikato_09MAR2016.tif,92.307692,RL,1,OK,True,True,0.534714


In [20]:
RL_Waikato.to_csv(prefix+"Nick/RL_Waikato.csv", index=False)

In [21]:
df.to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

### For those perfect matches, get the metadata from the corresponding matched image

In [22]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return {}
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return {}
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/1896 [00:00<?, ?it/s]

/home/nyou045/.local/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,compress,interleave,GCPs,res,photometric
0,MaxarImagery/HighFreq/Southland/ColacBay/Shorelines/ColacBay_13MAR2020.shp,MaxarImagery/HighFreq/Southland/ColacBay/Imagery/Stack/ColacBay_13MAR2020.tif,100.0,MAX,7,True,True,True,GTiff,uint8,NaN,13572,7762,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5, 0.0, 1205815.5010279878, 0.0, -0.5, 4854130.001446708, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,0,"(0.5, 0.5)",NaN
1,Retrolens/Northland/Kaikai/Shorelines/Kaikai_06JAN1978.shp,Retrolens/Northland/Kaikai/Stack/Kaikai_06JAN1978.tif,100.0,RL,22,True,True,True,GTiff,uint16,65535.0,56872,73163,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.14946675909999765, 0.0, 1638631.2033818986, 0.0, -0.14946675909999765, 6059358.759612814, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,0,"(0.14946675909999765, 0.14946675909999765)",NaN
2,MaxarImagery/HighFreq/Southland/ColacBay/Shorelines/ColacBay_22FEB2016.shp,MaxarImagery/HighFreq/Southland/ColacBay/Imagery/Stack/ColacBay_22FEB2016.tif,100.0,MAX,6,True,True,True,GTiff,uint8,NaN,17526,10304,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.37658312916755676, 0.0, 1205815.2866418455, 0.0, -0.37658312916755676, 4854129.708500402, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,0,"(0.37658312916755676, 0.37658312916755676)",NaN
3,Retrolens/Northland/Kaikai/Shorelines/Kaikai_29JAN1969.shp,Retrolens/Northland/Kaikai/Stack/Kaikai_29JAN1969.tif,100.0,RL,11,True,True,True,GTiff,uint16,65535.0,17586,23318,1,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.9035688153999958, 0.0, 1636296.0733494374, 0.0, -0.9035688153999958, 6062703.385285318, 0.0, 0.0, 1.0)",128.0,128,True,lzw,band,0,"(0.9035688153999958, 0.9035688153999958)",NaN
4,Retrolens/Northland/WhareanaBay/Shorelines/WhareanaBay_05JAN1980.shp,Retrolens/Northland/WhareanaBay/Stack/WhareanaBay_05JAN1980.tif,100.0,RL,12,True,True,True,GTiff,uint16,65535.0,19668,30709,1,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.45093247869999936, 0.0, 1596233.890845498, 0.0, -0.45093247869999936, 6193539.217827604, 0.0, 0.0, 1.0)",128.0,128,True,lzw,band,0,"(0.45093247869999936, 0.45093247869999936)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,Retrolens/Auckland/LongBay/Shorelines/LongBay_20MAR1988.shp,Retrolens/Auckland/LongBay/Stack/LongBay_20MAR1988_mosaic.jp2,100.0,RL,1,True,True,True,JP2OpenJPEG,uint16,256.0,2761,8352,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.6999999999999831, 0.0, 1755606.9976584367, 0.0, -0.7000000000000446, 5941457.707095626, 0.0, 0.0, 1.0)",1024.0,1024,True,NaN,pixel,0,"(0.6999999999999831, 0.7000000000000446)",NaN
1892,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_01JAN2015.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_01JAN2015.tif,100.0,MAX,8,True,True,True,GTiff,uint8,NaN,16541,2693,3,None,"(0.5857468398953292, 0.0, 1703895.3873867823, 0.0, -0.585746839895327, 5684779.542664661, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,3,"(0.5857468398953292, 0.585746839895327)",NaN
1893,MaxarImagery/HighFreq/Taranaki/Waitara/Shorelines/Waitara_28JAN2022.shp,MaxarImagery/HighFreq/Taranaki/Waitara/Imagery/Stack/Waitara_28JAN2022.tif,100.0,MAX,9,True,True,True,GTiff,uint8,NaN,20821,3390,3,None,"(0.4653431406149987, 0.0, 1703894.949276844, 0.0, -0.46534314061484816, 5684779.62871439, 0.0, 0.0, 1.0)",128.0,128,True,lzw,pixel,3,"(0.4653431406149987, 0.46534314061484816)",NaN
1894,Retrolens/Auckland/LongBay/Shorelines/LongBay_19JAN1973.shp,Retrolens/Auckland/LongBay/Stack/LongBay_19JAN1973_mosaic.jp2,100.0,RL,2,True,True,True,JP2OpenJPEG,uint16,256.0,4758,14567,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.3999999999999902, 0.0, 1755628.652892

## Read LDS index tiles

In [23]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$") | filelist.str.endswith("Canterbury_2022.shp")]
index_tiles

4529     Archive/Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10090            Archive/Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11967         Archive/Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13951         Archive/Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14070              Archive/Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                                   ...                                                           
36402                     SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
36407                SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-

In [24]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/80 [00:00<?, ?it/s]

148967

In [25]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   22679
2022                   12908
2023                    3607
24/02/2017              2685
                       ...  
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
Name: count, Length: 384, dtype: Int64

## LDS index tile matching

In [26]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30742,Archive/Gabrielle/Shorelines/Hawkes Bay/Napier/Heretaunga_20FEB2023.shp,Retrolens/WestCoast/WhareateaRiver/Stack/WhareateaRiver_23FEB2023.tif,61.904762,LDS,9,OK,True,True
100392,Retrolens/Gisborne/Tuparoa/Shorelines/Tuparoa_08FEB2023.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_20FEB2016.tif,62.500000,LDS,11,OK,True,True
30213,Archive/Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,MaxarImagery/HighFreq/BayOfPlenty/Ohiwa/Imagery/Stack/Ohiwa_08APR2013.tif,62.608696,LDS,6,OK,True,True
30285,Archive/Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_Papamoa_04APR2023.shp,MaxarImagery/HighFreq/BayOfPlenty/Ohiwa/Imagery/Stack/Ohiwa_08APR2013.tif,62.608696,LDS,7,OK,True,True
102436,Retrolens/Gisborne/Waiparapara/Shorelines/Waiparapara_07MAR2023.shp,MaxarImagery/HighFreq/Gisborne/WaipiroBay/Imagery/Stack/WaipiroBay_22MAR2003.tif,63.157895,LDS,6,OK,True,True
...,...,...,...,...,...,...,...,...
40746,Retrolens/Southland/Tiwai_Point/Shorelines/TiwaiPoint_07FEB2016.shp,MaxarImagery/HighFreq/Southland/TiwaiPoint/Stack/TiwaiPoint_02FEB2016.tif,94.444444,LDS,1,OK,True,True
60127,Retrolens/Manawatu/HimatangiBeach_North/Shorelines/HimatangiBeach_North_02FEB2021.shp,Retrolens/Manawatu/HimatangiBeach_North/Stack/HimatangiBeach_North_21FEB2001_mosaic.jp2,94.545455,LDS,3,OK,True,True
35612,Retrolens/Waikato/Pauanui_Tairua/Shorelines/Pauanui_Tairua_04FEB2019.shp,MaxarImagery/HighFreq/Waikato/Pauanui_Tairua/Imagery/Stack/Pauanui_Tairua_09FEB2019.tif,95.238095,LDS,4,OK,True,True
49225,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Stack/Oakura_03OCT2016.tif,96.551724,LDS,3,OK,True,True


In [27]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured", "FLOWN1", "FLOWN2"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    elif "southland-075m-rural-aerial-photos-index-tiles-2005-2011.shp" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .75
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
print("Has Pixel_ER: ", sum(~LDS.Pixel_ER.isna()))
print("No Pixel_ER: ", sum(LDS.Pixel_ER.isna()))
LDS

  0%|          | 0/521 [00:00<?, ?it/s]

Perfect matches: 391
Imperfect matches: 130
Has Pixel_ER:  521
No Pixel_ER:  0


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
243,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
180,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
330,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Shorelines/OpihiRiver_RangitataRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
293,Retrolens/Canterbury/RakaiaSouth/Shorelines/RakaiaSouth_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
249,Retrolens/Canterbury/OpihiRiver_OrariRiver/Shorelines/OpihiRiver_OrariRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
...,...,...,...,...,...,...,...,...
237,Retrolens/Auckland/OnetangiBay/Shorelines/OnetangiBay_25APR2017.shp,"[Retrolens/Auckland/Whatipu/lds/auckland-0075m-urban-aerial-photos-index-tiles-2017/auckland-0075m-urban-aerial-photos-index-tiles-2017.shp, SpatialData/Mosaics/Auckland 2017/Imagery/auckland-0075...",2017-04-25,25/04/2017,2017-04-25,FLOWN,100.000000,0.075
68,Retrolens/Northland/TaupoBay/Shorelines/TaupoBay_03JUN2016.shp,[SpatialData/Mosaics/Footprint shapefiles/Northland Footrpints/northland-04m-rural-aerial-photos-index-tiles-2014-2016.shp],2016-06-03,3/06/2016,"03/06/2016, 10/11/2014, 30/03/2016",DATE_DMY,100.000000,0.4
235,Retrolens/Bay of Plenty/Torere_Hawai/Shorelines/Torere_Hawai_20DEC2021.shp,"[Archive/Gabrielle/Imagery/pre-storm/LINZ/BayofPlenty/LINZ20212022/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021-2022/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021-2022.shp, Arch...",2021-12-20,20/12/2021,20211220,FLOWN,100.000000,0.2
64,Retrolens/Gisborne/Whangara/Shorelines/Whangara_16FEB2022.shp,"[Archive/Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photos-index-tiles-2021-2023Copy.shp, Archive/Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-ru...",2022-02-16,16/02/2022,"16/02/2022, 18/12/2021",FLOWN,100.000000,0.2


In [28]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
243,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
180,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
330,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Shorelines/OpihiRiver_RangitataRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
293,Retrolens/Canterbury/RakaiaSouth/Shorelines/RakaiaSouth_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
249,Retrolens/Canterbury/OpihiRiver_OrariRiver/Shorelines/OpihiRiver_OrariRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
131,Retrolens/Canterbury/OpihiRiver_South/Shorelines/OpihiRiver_South_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
325,Retrolens/Canterbury/AshburtonRiver_North/Shorelines/AshburtonRiver_North_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,2/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
467,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
441,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
341,Retrolens/HawkesBay/OceanBeach/Shorelines/OceanBeach_06MAR2019.shp,[SpatialData/Mosaics/Footprint shapefiles/Hawkes Bay Footprints/hawkes-bay-03m-rural-aerial-photos-index-tiles-2019-2020.shp],2019-03-06,6/03/2019,05/03/2019,FLOWN,44.444444,0.3


In [29]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)